In [1]:
# 라이브러리 임포트
import os
from pymongo import MongoClient
from pyproj import Transformer


In [2]:
# MongoDB 연결하기
client = MongoClient('@@@',
          username='@@@',
          password='@@@',
          authSource='admin',
          authMechanism='SCRAM-SHA-256')

database = client.get_database('Wedrive2024')

# 사용자 정보
collection_user = database.get_collection('user_coordinate_area')
users=collection_user.find()

# 음식점 정보
collection_public = database.get_collection('raw_public_05')
restaurants=collection_user.find()

# 새로 만들 컬렐션
collection_new = database.get_collection('User_destination_restaurant')

In [3]:
# 위성좌표변환 (EPSG:5174 (Korea 2000 / Unified CS) -> EPSG:4326 (WGS84))
transformer = Transformer.from_crs("EPSG:5174", "EPSG:4326", always_xy=True)
def transformer_to_epsg4326(x, y):              #x,y는 좌표정보(5174)
    lon, lat = transformer.transform(x, y)
    return [round(lat,4),round(lon,4)]

In [ ]:
# collection_user에서 좌표에 해당하는 값을 collection_restaurant에서 확인하여 존재하면 DB에 삽입
for user in users:
    user_a, user_b= round(users[0]['destination_lat'], 4), round(users[0]['destination_lng'],4)
    for restaurant in restaurants:
        re_a, re_b=transformer_to_epsg4326(restaurant['좌표정보x(epsg5174)'], restaurant['좌표정보y(epsg5174)'])
        if user_a==re_a and user_b==re_b:
            docs=user

In [5]:
a, b= round(users[0]['destination_lat'], 4), round(users[0]['destination_lng'],4)
a, b

(35.8143, 128.5262)